# Question 2 - VaR Calculations (Linear and Quadratic Approximations)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

## Initialize Variables

In [40]:
#Call option
So1= 50         #Intial Value for stock 1
K1 = 51         #Strike Price 
sig1 = 0.28     #Volatility 
T1 = 9/12       #Time to Maturity

#Put Option
So2= 20         #Intial Value for stock 1
K2 = 19         #Strike Price 
sig2 = 0.25     #Volatility 
T2 = 1          #Time to Maturity 

r = 0.06        #Risk free rate
rho = 0.4       #Correlation
N =10000000     #Simulations 


## Main Code Block 

### Part a - Linear Approximation(Delta Approach) 

In [46]:
#Delta for call
d11 = (np.log(So1/K1) + (r+sig1**2/2)*T1)/(sig1*T1**0.5) 
deltac = st.norm.cdf(d11)

#Delta for put
d12 = (np.log(So2/K2) + (r+sig2**2/2)*T2)/(sig2*T2**0.5) 
deltap = st.norm.cdf(d12)-1

#Random variable generation
z1 = np.random.normal(0,1,N)
z2 = rho * z1 + (1-rho**2)**0.5 * np.random.normal(0,1,N)

#Changes in stock prices
delS1 = So1 * sig1 * z1 / (252**0.5)
delS2 = So2 * sig2 * z2 / (252**0.5)

#Changes in Portfolio 
delV = delS1*deltac + delS2*deltap

print("99% 10 day VaR for portfolio = ",-np.quantile(delV,0.01)*(10**0.5))

99% 10 day VaR for portfolio =  3.6086919398436743


### Part b - Quadratic Approximation (Delta Gamma Approach)

In [47]:
#Delta and Gamma for call
d11 = (np.log(So1/K1) + (r+sig1**2/2)*T1)/(sig1*T1**0.5) 
deltac = st.norm.cdf(d11)
Gammac = st.norm.pdf(d11)/(So1*sig1*(T1**0.5))

#Delta and Gamma for put
d12 = (np.log(So2/K2) + (r+sig2**2/2)*T2)/(sig2*T2**0.5) 
deltap = st.norm.cdf(d12)-1
Gammap = st.norm.pdf(d12)/(So2*sig2*(T2**0.5))

#Random variable generation
z1 = np.random.normal(0,1,N)
z2 = rho * z1 + (1-rho**2)**0.5 * np.random.normal(0,1,N)

#Changes in stock prices
delS1 = So1 * sig1 * z1 / (252**0.5)
delS2 = So2 * sig2 * z2 / (252**0.5)

#Changes in Portfolio 
delV = delS1*deltac + delS2*deltap + 0.5 * (Gammac * delS1**2 + Gammap * delS2**2)

print("99% 10 day VaR for portfolio = ",-np.quantile(delV,0.01)*(10**0.5))

99% 10 day VaR for portfolio =  3.388476725716707


### Part c - Full Simulation

In [48]:
#Delta and Gamma for call
d1 = lambda X,sig,k,t: (np.log(X/k) + (r+sig**2/2)*t)/(sig*t**0.5) 

#initial Value of Call and put 
C0 = So1 * st.norm.cdf(d1(So1,sig1,K1,T1)) - (K1*np.exp(-r*T1)*st.norm.cdf(d1(So1,sig1,K1,T1)-(sig1*T1**0.5)))
P0 = -So2 * st.norm.cdf(-d1(So2,sig2,K2,T2)) + (K2*np.exp(-r*T2)*st.norm.cdf(-d1(So2,sig2,K2,T2)+(sig2*T2**0.5)))
Vo = C0+P0

#Simulate N values for Stock price 
#Random variable generation
z1 = np.random.normal(0,1,N)
z2 = rho * z1 + (1-rho**2)**0.5 * np.random.normal(0,1,N)

h = 1/252
S1 = So1 * np.exp((r - (sig1**2)/2)*h + sig1 * h**0.5 * z1)
S2 = So2 * np.exp((r - (sig2**2)/2)*h + sig2 * h**0.5 * z2)

#New Call and Put Prices
C1 = S1 * st.norm.cdf(d1(S1,sig1,K1,T1)) - (K1*np.exp(-r*T1)*st.norm.cdf(d1(S1,sig1,K1,T1)-(sig1*T1**0.5)))
P1 = -S2 * st.norm.cdf(-d1(S2,sig2,K2,T2)) + (K2*np.exp(-r*T2)*st.norm.cdf(-d1(S2,sig2,K2,T2)+(sig2*T2**0.5)))
V1 = C1+P1

delV = V1 - Vo
print("99% 10 day VaR for portfolio = ",-np.quantile(delV,0.01)*(10**0.5))

99% 10 day VaR for portfolio =  3.313488410600105
